<a href="https://colab.research.google.com/github/rudycav/RedditScraper/blob/main/Project/r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [379]:
!pip install asyncpraw
!pip install praw
!pip install nltk
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [380]:
import asyncpraw
import praw
from praw.models import MoreComments
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
import string
from string import punctuation
from emoji import demojize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [381]:
reddit = praw.Reddit(
    client_id="BE3T0CMpNrtqLT97yL6XiA",
    client_secret = "gJnvUXTK0gCnj3UMGSUtw0JW35tN1A",
    password = "Hannah00!!@",
    user_agent = "spidey by u/rudyboycav",
    username = "rudyboycav",
)

In [382]:
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for x in hot_topic: 
    c = x.comments.list()
    for y in c:
      if isinstance(y, MoreComments):
        continue
      df = df.append({#'subreddit': subreddit_page,
                      #'id': x.id,
                      #'title': x.title,
                      #'author': x.author,
                      'comments': y.body},
                      ignore_index=True) 

  return df

In [383]:
wall = reddit_scraper(reddit, 'wallstreetbets', 2)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [384]:
wall.head()

,comments
0,I should have stayed cash.
1,Lol how the fuck is 2022 almost over
2,CNBC: Futures little changed. \n\nWSB: REEEEEE...
3,"I wish this sub existed back in 2008, I‘d real..."
4,corn better be 15K by open


In [385]:
for x in wall.comments:
  print(x)

I should have stayed cash.
Lol how the fuck is 2022 almost over
CNBC: Futures little changed. 

WSB: REEEEEEEEEEEEEEEEEEEE.
I wish this sub existed back in 2008, I‘d really like to read some of the conversations back then everything dumped
corn better be 15K by open
Everyone is expecting market to sell off ![img](emote|t5_2th52|12787)
45 mins til a big falling knife ![img](emote|t5_2th52|18632)![img](emote|t5_2th52|4271)![img](emote|t5_2th52|4641)![img](emote|t5_2th52|4887)
I'm not an alcoholic, I just have to drink every day to avoid the shakes
All ye who survive this market will be millionaires next bull market 📈
This guy is the worst poster on WSB ⬇️
Shitcoins dumping furiously. Good luck tomorrow bulls ![img](emote|t5_2th52|4267)
Imagine thinking companies with a 30+ p/e with shrinking earnings, rising interest rates, and global recession is a great buy
I'm starting a petition to bring back pointing Morgan Freeman in the emojis.
Real OG's remember when you could get fat as FUCK in 

In [386]:
def clean_text(df):
  clean_text = " ".join(str(df).split())
  clean_text = " ".join([w for w in df.split() if w.isalpha()])

  return clean_text

gg = wall.comments.map(clean_text)

In [387]:
gg

0                                   I should have stayed
1                        Lol how the fuck is almost over
2                                         Futures little
3      I wish this sub existed back in really like to...
4                                 corn better be by open
                             ...                        
515    other communities The ridiculous thing is that...
516                                            a spot on
517    I got permabanned from there for pumping bed l...
518    Dah not I just went all in on popcorn futures ...
519    To be clear not referring to you OIU or any ot...
Name: comments, Length: 520, dtype: object

In [388]:
def word_tokenizer(df):
  tokens = word_tokenize(str(df))

  return tokens

ggg =word_tokenizer(gg)

In [389]:
from nltk.corpus import stopwords
def stopwords(df):
  stopwords_ = set(stopwords.words("english"))
  clean_tokens = [t for t in df if not t in stopwords_]
  clean_text = " ".join(clean_tokens)

  return clean_text

stopwords(ggg)

AttributeError: ignored

In [ ]:
non_letter = "".join([x for x in str(cleaned_text) if x not in string.punctuation])
non_letter

In [ ]:
def remove_punctuation(df):

  #non_letter = re.sub(r"https?://\S+", "", str(df))
  non_letter = "".join([x for x in str(df) if x not in string.punctuation]) #removes punctuation
  non_letter = ''.join([i for i in str(df) if not i.isdigit()]) #removes numbers
  #non_letter = demojize(str(df))
  
  return non_letter

cleaned_text = remove_punctuation(cleaned_text)

In [ ]:
cleaned_text

In [ ]:
for x in cleaned_text:
  if x is 'SPY':
    print(x)

In [ ]:
for x in cleaned_text:
  if x.isupper():
    print(x)

In [ ]:
def tokenize(df):
  for words in wall:
    tokenize = word_tokenize(words)
  return tokenize

tokenized_words = tokenize(wall)

In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
def stopwords(df):
  #stop_words = set(stopwords.words("english"))
  filtered_words = [word for word in df if word.casefold() not in stop_words]
  return filtered_words

filtered_stopwords = stopwords(tokenized_words)
filtered_stopwords

In [ ]:
def stemming(df):
  stemmed_list = []
  snow_stemmer = SnowballStemmer(language='english')
  
  for words in filtered_stopwords:
    stemmer = snow_stemmer.stem(words)
    stemmed_list.append(stemmer)
  return stemmed_list

stemmed_words = stemming(filtered_stopwords)
stemmed_words

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets")
async for x in subreddit.hot(limit=3):
    print('THREAD: {}'.format(x.title))
    print('USER: {}'.format(x.author))

    print(x.comments)
    

